In [19]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
class Trajectory:
    
    def __init__(self, dendros):
        self.dendros = dendros

class Tracker:
    
    def __init__(self, dendrograms):
        self.frames = dendrograms
        self.trajectories = []
    
    def add_traj(self):
        # if wanna use node above used node, merge up or delete old node
        # if wanna use node below, split down or merge up or delete
        # momentum conservation needs actual trajectory merges
        # init is all leaves are own traj
        # jump moves only?
        # regularizer should penalize short trajectories

In [ ]:
def e_dyn(model, b):
    # traj in pixels. +1 for discretization error and
    # prevent divide by zero. vectorized.
    E = 0
    for traj in model.trajectories:
        locs = np.array([d.loc for d in traj.dendros])
        v = np.diff(locs, axis=0)
        m = np.array([d.mass for d in traj.dendros])[:-1]
        p = m*v
        p_norm = np.linalg.norm(p, axis=-1) + 1
        p1 = p[:-1]
        p2 = p[1:]
        p1_norm = p_norm[:-1]
        p2_norm = p_norm[1:]
        angle_term = -2 * (p1*p2).sum(1)/(p1_norm*p2_norm)
        inertia_term = p1_norm/p2_norm + p2_norm/p1_norm
        E += np.sum(angle_term + b*inertia_term)
    return E

def e_pers(model, d):
    E = 0
    for traj in model.trajectories:
        b1 = traj.dendros[0].loc
        endx, endy = traj.dendros[-1].loc
        b2 = min(np.abs(endx-d), np.abs(endy-d))
        E += scipy.special.expit(b1-1) + scipy.special.expit(b2-1)
    return E

def e_reg(model):
    return sum([1/len(traj) for traj, _, _ in model]) + len(model)

def ident(model):
    return 0    

def energy(model):
    a = 1
    b = 1
    c = 1
    d = 1
    e = 1
    return e_obs(model) + a*e_dyn(model, b) + c*e_pers(model)
        + d*e_reg(model) + e*ident(model)